In [1]:
import pandas as pd
import geopandas

In [2]:
county_csv = pd.read_csv('src/data/mn_demographics_history.csv')
county_csv.columns

Index(['statefp', 'county', 'fips', 'population2018', 'classification',
       'dVotes_2012', 'dPct_2012', 'rVotes_2012', 'rPct_2012', 'votes_2012',
       'd_two_party_2012', 'r_two_party_2012', 'partisanVotes_2012',
       'winner_2012', 'dVotes_2016', 'dPct_2016', 'rVotes_2016', 'rPct_2016',
       'votes_2016', 'd_two_party_2016', 'r_two_party_2016',
       'partisanVotes_2016', 'winner_2016', 'flipped', 'cpvi_score',
       'cpvi_rating', 'turnout_2012', 'turnout_2016', 'turnout_2018',
       'd_gov_2018', 'd_gov_pct_2018', 'r_gov_2018', 'r_gov_pct_2018',
       'votes_gov_2018', 'winner_2018'],
      dtype='object')

In [3]:
# Bring in TIGER shapefile for data
county_tiger = geopandas.read_file('/Users/Coreymj/Documents/GIS/usa/tl_2019_us_county/tl_2019_us_county.shp')

In [4]:
county_tiger_mn = county_tiger[county_tiger['STATEFP'] == '27'].copy()
county_tiger_mn.GEOID = county_tiger_mn.GEOID.astype('int64')
county_tiger_mn.sort_values(['NAME']).head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
289,27,001,00663198,27001,Aitkin,Aitkin County,06,H1,G4020,None,None,None,A,4718433588,449373340,+46.6024464,-093.4197600,"POLYGON ((-93.77799 46.59493, -93.77799 46.595..."
1612,27,003,00659447,27003,Anoka,Anoka County,06,H1,G4020,378,33460,None,A,1092880512,62442714,+45.2741098,-093.2427232,"POLYGON ((-93.51099 45.29935, -93.51097 45.300..."
772,27,005,00659448,27005,Becker,Becker County,06,H1,G4020,None,None,None,A,3406052328,336954677,+46.9376289,-095.7417568,"POLYGON ((-96.17430 46.89165, -96.17430 46.891..."
905,27,007,00659449,27007,Beltrami,Beltrami County,06,H1,G4020,None,13420,None,A,6487154450,1426287781,+47.8795077,-095.0050500,"POLYGON ((-95.59382 48.19455, -95.59382 48.194..."
1705,27,009,00659450,27009,Benton,Benton County,06,H1,G4020,378,41060,None,A,1057483359,12286988,+45.7012249,-094.0014397,"POLYGON ((-94.24770 45.73522, -94.24784 45.735..."


In [5]:
county_out = county_csv.copy()[[
    'statefp',
    'county',
    'fips',
    'population2018',
    'classification',
    'dVotes_2016',
    'dPct_2016',
    'rPct_2016',
]]

county_out.fips = county_out.fips.astype('int64')
county_out.head()

,statefp,county,fips,population2018,classification,dVotes_2016,dPct_2016,rPct_2016
0,27,Aitkin,27001,15896,rural,3134,0.341,0.601
1,27,Anoka,27003,357851,suburban,75499,0.410,0.507
2,27,Becker,27005,34420,rural,5208,0.305,0.637
3,27,Beltrami,27007,46834,rural,8688,0.408,0.506
4,27,Benton,27009,40452,rural,5640,0.283,0.646


In [6]:
county_tiger_mn[[
        'GEOID',
        'ALAND'
    ]].sort_values(['GEOID'])

,GEOID,ALAND
289,27001,4718433588
1612,27003,1092880512
772,27005,3406052328
905,27007,6487154450
1705,27009,1057483359
...,...,...
620,27165,1126526388
1672,27167,1945079889
878,27169,1621638207
633,27171,1712289572


In [7]:
county_out = county_out.merge(
    county_tiger_mn[[
        'GEOID',
        'ALAND'
    ]],
    how="left",
    left_on="fips",
    right_on="GEOID"
)
county_out.head()

,statefp,county,fips,population2018,classification,dVotes_2016,dPct_2016,rPct_2016,GEOID,ALAND
0,27,Aitkin,27001,15896,rural,3134,0.341,0.601,27001,4718433588
1,27,Anoka,27003,357851,suburban,75499,0.410,0.507,27003,1092880512
2,27,Becker,27005,34420,rural,5208,0.305,0.637,27005,3406052328
3,27,Beltrami,27007,46834,rural,8688,0.408,0.506,27007,6487154450
4,27,Benton,27009,40452,rural,5640,0.283,0.646,27009,1057483359


In [8]:
county_out['pop_density_2018'] = county_out['population2018'] / (county_out['ALAND'] / 2589988.1103)
county_out.sort_values('pop_density_2018', ascending=False).head()

,statefp,county,fips,population2018,classification,dVotes_2016,dPct_2016,rPct_2016,GEOID,ALAND,pop_density_2018
84,27,Ramsey,27123,552232,urban,176929,0.657,0.263,27123,394308034,3627.301984
81,27,Hennepin,27053,1261104,urban,429283,0.638,0.285,27053,1434450857,2276.999836
1,27,Anoka,27003,357851,suburban,75499,0.410,0.507,27003,1092880512,848.061453
80,27,Dakota,27037,428558,suburban,110251,0.482,0.436,27037,1456983368,761.820724
86,27,Washington,27163,261512,suburban,67086,0.470,0.451,27163,995618866,680.293427


In [9]:
mn_incomes = pd.read_csv('src/data/countyMHHI2018.csv')
mn_incomes.head()

,geoid,name,estimate,moe
0,27001,"Aitkin County, Minnesota",45989,2244
1,27003,"Anoka County, Minnesota",79923,1132
2,27005,"Becker County, Minnesota",59278,1882
3,27007,"Beltrami County, Minnesota",47410,2260
4,27009,"Benton County, Minnesota",56357,2775


In [10]:
county_out = county_out.merge(
    mn_incomes[[
        'geoid',
        'estimate'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid"
)
county_out.rename(columns={'estimate': 'median_income'}, inplace=True)

In [11]:
mn_age = pd.read_csv('src/data/acs2018_5yr_B01002_05000US27079.csv')
mn_age['geoid_split'] = mn_age['geoid'].str.split("US").str[1]
mn_age.geoid_split = mn_age.geoid_split.astype('int64')
mn_age.rename(columns={'B01002001': 'median_age'}, inplace=True)
mn_age.head()

,geoid,name,B01002000.5,"B01002000.5, Error",median_age,"B01002001, Error",B01002002,"B01002002, Error",B01002003,"B01002003, Error",geoid_split
0,04000US27,Minnesota,NaN,NaN,37.9,0.1,37.0,0.2,38.9,0.1,27
1,05000US27001,"Aitkin County, MN",NaN,NaN,55.1,0.2,54.8,0.3,55.4,0.3,27001
2,05000US27003,"Anoka County, MN",NaN,NaN,38.4,0.2,37.7,0.2,39.3,0.3,27003
3,05000US27005,"Becker County, MN",NaN,NaN,41.8,0.4,41.9,0.5,41.5,0.5,27005
4,05000US27007,"Beltrami County, MN",NaN,NaN,33.7,0.2,32.7,0.5,35.0,0.4,27007


In [12]:
county_out = county_out.merge(
    mn_age[[
        'geoid_split',
        'median_age'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid_split"
)

In [13]:
mn_race = pd.read_csv('src/data/acs2018_5yr_B03002_05000US27079.csv')
mn_race['geoid_split'] = mn_race['geoid'].str.split("US").str[1]
mn_race.geoid_split = mn_race.geoid_split.astype('int64')
mn_race.rename(columns={'B03002001': 'total_pop', 'B03002003': 'not_hispanic_white_alone'}, inplace=True)
mn_race['pct_nonwhite'] = 1 - (mn_race.not_hispanic_white_alone / mn_race.total_pop)
mn_race.head()


,geoid,name,total_pop,"B03002001, Error",B03002002,"B03002002, Error",not_hispanic_white_alone,"B03002003, Error",B03002004,"B03002004, Error",...,B03002018,"B03002018, Error",B03002019,"B03002019, Error",B03002020,"B03002020, Error",B03002021,"B03002021, Error",geoid_split,pct_nonwhite
0,04000US27,Minnesota,5527358,0,5234594,77,4438071,845,336505,2742,...,90812,3448,20534,1228,10023,866,10511,1005,27,0.197072
1,05000US27001,"Aitkin County, MN",15834,0,15620,0,14878,4,84,33,...,32,33,34,26,15,20,19,19,27001,0.060376
2,05000US27003,"Anoka County, MN",347431,0,332101,0,284994,268,20216,554,...,5153,734,1871,385,1040,278,831,270,27003,0.179711
3,05000US27005,"Becker County, MN",33773,0,33099,0,29190,16,184,48,...,82,48,90,50,48,31,42,42,27005,0.135700
4,05000US27007,"Beltrami County, MN",46117,0,45053,0,33283,33,496,41,...,28,28,160,75,48,43,112,54,27007,0.278292


In [14]:
county_out = county_out.merge(
    mn_race[[
        'geoid_split',
        'pct_nonwhite'
    ]],
    how="left",
    left_on="fips",
    right_on="geoid_split"
)

In [15]:
county_out.to_json('src/data/mn_demographics_history.json', orient='records')
county_out.head(25)

,statefp,county,fips,population2018,classification,dVotes_2016,dPct_2016,rPct_2016,GEOID,ALAND,pop_density_2018,geoid,median_income,geoid_split_x,median_age,geoid_split_y,pct_nonwhite
0,27,Aitkin,27001,15896,rural,3134,0.341,0.601,27001,4718433588,8.725449,27001,45989,27001,55.1,27001,0.060376
1,27,Anoka,27003,357851,suburban,75499,0.410,0.507,27003,1092880512,848.061453,27003,79923,27003,38.4,27003,0.179711
2,27,Becker,27005,34420,rural,5208,0.305,0.637,27005,3406052328,26.173230,27005,59278,27005,41.8,27005,0.135700
3,27,Beltrami,27007,46834,rural,8688,0.408,0.506,27007,6487154450,18.698415,27007,47410,27007,33.7,27007,0.278292
4,27,Benton,27009,40452,rural,5640,0.283,0.646,27009,1057483359,99.075033,27009,56357,27009,35.6,27009,0.093466
5,27,Big Stone,27011,5000,rural,921,0.337,0.589,27011,1292861685,10.016493,27011,51920,27011,48.9,27011,0.024721
6,27,Blue Earth,27013,67785,outstate city,14428,0.434,0.471,27013,1936751358,90.647849,27013,55861,27013,30.7,27013,0.116613
7,27,Brown,27015,25325,rural,3763,0.275,0.637,27015,1582765932,41.441029,27015,57728,27015,42.1,27015,0.063901
8,27,Carver,27019,105970,suburban,21514,0.390,0.527,27019,917364672,299.184227,27019,97895,27019,37.5,27019,0.107533
9,27,Cass,27021,29470,rural,4949,0.312,0.628,27021,5235592104,14.578475,27021,50173,27021,48.9,27021,0.169733
